Below, we pull in the CRISM data, clip the borders,  replace null values with 0, and layer L and S images. 

In [1]:
import pickle
import pandas as pd
import numpy as np
import spectral.io.envi as envi
from utils.constants import *
from preprocessing.CRISM_preprocess_helpers import *

Initialize file names: what CRISM file do you want to preprocess?

In [2]:
img_dir = "/Users/marina/mars_data/TRDR10628/processed/"

l_img_name = "frt00010628_07_if165l_trr3"

s_img_name = "frt00010628_07_if165s_trr3"

d_img_name = "frt00010628_07_de165s_ddr1"


CUR_IMG = "frt00010628"

CRISM_DIR = PREPROCESSED_DATA + "CRISM/" 


Init files (DDR, S, L). 
For frt00010628_07, do special case of dropping last band in L image, it is null.

In [3]:
ddr_img, s_img, l_img = get_images(img_dir, d_img_name, s_img_name, l_img_name)

# Special case: Last wavelength of l I/F CRISM img has a wavelength value of null, 
# 65535, so I drop it

l_img =  l_img[:,:,:-1]


Reduce image size to row range:0-450 and col range: 25-626
Original range, row max= 450, col max=640


Layer S and L images into single image, and save as Pickle.

In [5]:
#  Layer L and S images. 
pixel_dir = img_dir
img_save_name = "frt00010628"

layer_CRISM(s_img, l_img, pixel_dir, img_save_name)

### Model Adapatations

We need to make sure the CRISM data is properly adjusted w.r.t. the lab spectra used. 

Reduced wavelengths: RW_CRISM and RW_USGS.
CRISM data has a certain wavelengths, and USGS has different ones. So, we need to find the mutually compatible set. This is what reducing wavelengths is.

When running the model, it will use these reduced wavelengths for the CRISM data and USGS endmembers. Saved in PREPROCESSED_DATA/CRISM

In [3]:
save_USGS_wavelengths()

Replacing 3  missing/NULL mags with next available value to fill-in.
[  0   1 239]


In [4]:
CRISM_reduced, USGS_reduced = record_CRISM_USGS_reduced_wavelengths()


CRISM reduced from 544 to 273
USGS reduced from 480 to 273
RMSE between normalized wavelength vectors: 0.001151306508751759


## TO DO
need to calculate optical constants a
and need to set up angles for CRISM data

7. Calculate optical constants, k, for each endmember (in each wavelength) 

Usually we run this on the server/HPC because it takes a while (and can use many threads to infer k in different wavelengths at the same time).

In [ ]:
from preprocessing.estimatek import estimate_all_USGS_k
# True means to match CRISM wavelengths
estimate_all_USGS_k(True)

Visualizations below

In [ ]:
from spectral import imshow
from utils.plotting import *
bands = (10, 20, 50)
imshow(data=s_img, bands=bands)

imshow(data=l_img, bands=bands)

with open(DATA_DIR + "PREPROCESSED_DATA/CRISM/" + img_save_name + '.pickle', 'rb') as handle:
    img = pickle.load(handle)
plot_CRISM_img(img, DATA_DIR + "PREPROCESSED_DATA/CRISM/" + "frt00010628.pdf")